In [1]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import numpy as np
from keras.utils import np_utils

In [2]:
import os
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import requests

#importing dependencies related to nn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#importing dependencies related to image transformations
import torchvision
from torchvision import transforms
from PIL import Image

#importing dependencies related to data loading
from torchvision import datasets
from torch.utils.data import DataLoader

#importing Tensorboard for data visualization
from torch.utils.tensorboard import SummaryWriter

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'shufflenet_v2_x1_0', pretrained=True)

Using cache found in /home/suhruth/.cache/torch/hub/pytorch_vision_v0.6.0


In [5]:
model=model.to(device)

In [15]:
import cv2,os

data_path='/home/suhruth/Downloads/archive(1)/Face Mask Dataset/Train'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'WithMask': 0, 'WithoutMask': 1}
['WithMask', 'WithoutMask']
[0, 1]


In [16]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [17]:
from keras.utils import np_utils
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)
new_target=np_utils.to_categorical(target)

np.save('data',data)
np.save('target',new_target)

In [18]:
import numpy as np

data=np.load('/home/suhruth/Downloads/data.npy')
target=np.load('/home/suhruth/Downloads/target.npy')

In [19]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [21]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
31/31 [==============================] - 34s 1s/step - loss: 0.7745 - accuracy: 0.5133 - val_loss: 0.6643 - val_accuracy: 0.5524
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/20
31/31 [==============================] - 35s 1s/step - loss: 0.5752 - accuracy: 0.7207 - val_loss: 0.4947 - val_accuracy: 0.7782
INFO:tensorflow:Assets written to: model-002.model/assets
Epoch 3/20
31/31 [==============================] - 33s 1s/step - loss: 0.4009 - accuracy: 0.8094 - val_loss: 0.4615 - val_accuracy: 0.7944
INFO:tensorflow:Assets written to: model-003.model/assets
Epoch 4/20
31/31 [==============================] - 31s 1s/step - loss: 0.3256 - accuracy: 0.8692 - val_loss: 0.3191 - val_accuracy: 0.8589
INFO:tensorflow:Assets written to: model-004.model/assets
Epoch 5/20
31/31 [==============================] - 31s 1s/step - loss: 0.1911 - accuracy: 0.9226 - val_loss: 0.2760 - val_accuracy: 0.8992
INFO:tensorflow:Assets written to: model-005.model/assets
Epoch 6/20

In [22]:
yolo_config = '/home/suhruth/Documents/yolove3cfg.txt'
yolo_weights = '/home/suhruth/Downloads/yolov3.weights'
classes_file = '/home/suhruth/Documents/coco.names'

with open(classes_file,'r') as f:
    classes=[line.strip() for line in f.readlines()]
   
net=cv2.dnn.readNet(yolo_weights,yolo_config)

def humanDetect(image):
   
    blob=cv2.dnn.blobFromImage(image,1/255,(512,512),(0,0,0),True,crop=False)
    
    net.setInput(blob)
    layer_names=net.getLayerNames()
    output_layers=[layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    outs=net.forward(output_layers)
    class_ids= list()
    confidences= list()
    boxes= list()

    for out in outs:
        # iterate over anchor boxes for human class
        for detection in out:
            #bounding box
            center_x=int(detection[0] * image.shape[1])
            center_y=int(detection[1] * image.shape[0])
            w=int(detection[2] * image.shape[1])
            h=int(detection[3] * image.shape[0])
            x=center_x - w // 2
            y=center_y - h // 2
            boxes.append([x,y,w,h])
            
            #class
            class_id=np.argmax(detection[5:])
            class_ids.append(class_id)
            confidence=detection[4]
            confidences.append(float(confidence))
   
    #non-max supression
    ids=cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.3)
    #draw the bounding boxes on the image
    colors= np.random.uniform(0,255,size=(len(classes),3))

    imgs = []

    for i in ids:
        i=i[0]
        x,y,w,h=boxes[i]
        class_id=class_ids[i]
        color=colors[class_id]
        if classes[class_id]=='person':
            crop= image[y:y+h,x:x+w]
            imgs.append([crop, [x, y, w, h]])           
            # return [crop, [x, y, w, h]]

    if (len(imgs) == 0):
      return None
    else: 
      return imgs 

In [43]:
model = load_model('/home/suhruth/model-009.model')

face_clsfr=cv2.CascadeClassifier('/home/suhruth/Documents/haarcascade_frontalface_alt.xml')
cap=cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('/home/suhruth/Documents/finals.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 20, (frame_width,frame_height))
labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}
frame_index=0
while(True):
    ret,img=cap.read()
    if not ret:
        break
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  
    for (x,y,w,h) in faces:
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        label=result[0][1]
        if label >=0.5:
            label = 1
        else: label = 0
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        print(result,label)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    out.write(img)
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    if(key==27):
        break
cv2.destroyAllWindows()
cap.release()

[[0.97772425 0.02227579]] 0
[[0.9779874  0.02201259]] 0
[[0.9748669  0.02513304]] 0
[[0.9796652 0.0203348]] 0
[[0.961725   0.03827501]] 0
[[0.9716684  0.02833154]] 0
[[0.97397715 0.02602284]] 0
[[0.98449695 0.01550303]] 0
[[0.9675437  0.03245626]] 0
[[0.9613936  0.03860636]] 0
[[0.9416248  0.05837516]] 0
[[0.9632315  0.03676846]] 0
[[0.9648855  0.03511459]] 0
[[0.97121847 0.02878156]] 0
[[0.97396106 0.02603896]] 0
[[0.9785065  0.02149354]] 0
[[0.97830063 0.02169937]] 0
[[0.97067744 0.02932261]] 0
[[0.9852409  0.01475908]] 0
[[0.96846414 0.03153589]] 0
[[0.97558993 0.02441003]] 0
[[0.98164475 0.01835524]] 0
[[0.95954037 0.0404596 ]] 0
[[0.9692077  0.03079233]] 0
[[0.98099077 0.0190092 ]] 0
[[0.9800426  0.01995741]] 0
[[0.97995114 0.02004893]] 0
[[0.9737882  0.02621183]] 0
[[0.9737254  0.02627463]] 0
[[0.9739953  0.02600465]] 0
[[0.96762305 0.03237693]] 0
[[0.97623956 0.02376045]] 0
[[0.9753695  0.02463044]] 0
[[0.9690606  0.03093944]] 0
[[0.9678821  0.03211793]] 0
[[0.97123533 0.0287646